In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('..')

from backtest.performance_generater import PerformanceGenerator
from backtest.operators import *

In [2]:
open = pd.read_csv('/Volumes/Database/Factor_data/open.csv', index_col=0)
close = pd.read_csv('/Volumes/Database/Factor_data/close.csv', index_col=0)
returns = pd.read_csv('/Volumes/Database/Factor_data/return.csv', index_col=0)

# 要再確認一下 報酬正確性
unadj_return = close.pct_change().shift(-2) 
exp_returns = returns.shift(-2) * 0.01

# operatingmargin = pd.read_csv('/Volumes/Database/Factor_data/營業利益率_90.csv', index_col=0) * 0.01
# pb_q = pd.read_csv('/Volumes/Database/Factor_data/當季季底PB_90.csv', index_col=0)
# revenue_m = pd.read_csv('/Volumes/Database/Factor_data/以合併為主單月營收(千元)_90.csv', index_col=0)
operatingincome = pd.read_csv('/Volumes/Database/Factor_data/營業利益_90.csv', index_col=0)
# operatingincome = pd.read_csv('/Volumes/Database/Factor_data/單月營業利益(千元)_90.csv', index_col=0)
# outstanding = pd.read_csv('/Volumes/Database/Factor_data/流通在外股數(千股).csv', index_col=0)
# CR = pd.read_csv('/Volumes/Database/Factor_data/流動比率_90.csv', index_col=0)
# CD= pd.read_csv('/Volumes/Database/Factor_data/流動負債_90.csv', index_col=0)
# CA = pd.read_csv('/Volumes/Database/Factor_data/流動資產_90.csv', index_col=0)
# inv = pd.read_csv('/Volumes/Database/Factor_data/  存貨_90.csv', index_col=0)
asset = pd.read_csv('/Volumes/Database/Factor_data/資產總額_90.csv', index_col=0)
# netasset = pd.read_csv('/Volumes/Database/Factor_data/淨值資產_90.csv', index_col=0)
# netdebt = pd.read_csv('/Volumes/Database/Factor_data/淨負債_90.csv', index_col=0)
# NI = pd.read_csv('/Volumes/Database/Factor_data/稀釋稅後淨利_90.csv', index_col=0)
# revenue_yoy = pd.read_csv('/Volumes/Database/Factor_data/單月營收成長率％_90.csv', index_col=0)


/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_8457/1845704117.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  unadj_return = close.pct_change().shift(-2)


In [3]:
ZTXA_close = pd.read_csv('/Volumes/Database/Factor_data/ZXTA_close.csv', index_col=0)
ZTXA_close.index = pd.to_datetime(ZTXA_close.index)
market_returns = ZTXA_close['ZTXA'].pct_change()
Benchmark = ZTXA_close['ZTXA'].pct_change().shift(-2)

In [4]:
factor = rolling_regression(operatingincome/asset, returns, 315, 'beta')

# no fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     buy_fee = 0, 
                                                     sell_fee = 0,
                                                     benchmark = Benchmark
                                                     ).backtest()

# fee
returns_by_period, summary_df = PerformanceGenerator(factor = factor,
                                                     expreturn = exp_returns,
                                                     strategy='LO',
                                                     benchmark = Benchmark
                                                     ).backtest()

                                 1101
2014-08-20    [5.021931133461308e-05]
2014-08-21    [6.863272518471773e-05]
2014-08-22   [0.00010762929966008934]
2014-08-25    [7.782453064306568e-05]
2014-08-26    [6.919563501464525e-05]
...                               ...
2024-03-05   [-0.0002154730142246768]
2024-03-06  [-0.00020100836983294348]
2024-03-07   [-0.0002135804506378944]
2024-03-08  [-0.00019003543753550882]
2024-03-11   [-0.0001695424391819006]

[2334 rows x 1 columns]
                                 1102
2014-08-20   [0.00016240200579929753]
2014-08-21   [0.00018043212450457856]
2014-08-22    [0.0001859220018632972]
2014-08-25    [0.0001738345301444455]
2014-08-26   [0.00015438131075134402]
...                               ...
2024-03-01  [-1.9189289020171533e-05]
2024-03-04   [-6.126363105525716e-06]
2024-03-05  [-1.9285065613216313e-06]
2024-03-06   [4.4725068796623705e-06]
2024-03-07   [2.0420799093659366e-06]

[2060 rows x 1 columns]
                                 1103


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 315 is different from 314)

In [ ]:
# csv_file = '/Volumes/Database/medina_alpha/EBITtototalassets.csv'
# factor.to_csv(csv_file)